<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Rojas


### **Link de repositorio de GitHub:** `https://github.com/sebarojasmorales`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 12/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [255]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Requirement already up-to-date: plotly in c:\programdata\anaconda3\lib\site-packages (5.4.0)


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [256]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [257]:
def custom_features(dataframe_in):
    df=pd.DataFrame()
    df_aux=dataframe_in.loc[:,['Customer ID','Invoice','InvoiceDate']].drop_duplicates().sort_values(['Customer ID','InvoiceDate']).reset_index(drop=True)
    df_aux['IVT']=(df_aux['InvoiceDate'][1:].reset_index(drop=True)-df_aux['InvoiceDate'][0:len(df_aux)-1]).dt.days
    df_aux2=df_aux.drop(df_aux['Customer ID'].drop_duplicates(keep='last').index) 
    df['Lenght']=df_aux2.groupby('Customer ID').sum()['IVT']
    df['Recency']=(df_aux['InvoiceDate'].max()-df_aux.groupby('Customer ID').max()['InvoiceDate']).dt.days+1
    df['Frequency']=df_aux.groupby('Customer ID').count()['Invoice']
    df['Monetary']=dataframe_in.groupby('Customer ID').sum()['Price']/df['Frequency']
    df['Periodicity']=df_aux2.groupby('Customer ID').std(0)['IVT']
    df=df[df['Frequency']>1]
    df=df.reset_index().rename(columns={'index':'Customer ID'})
    
    
    return df

# consideré que para que sean distintas compras deben tener distinto Invoice
# ocupé la std con ddof=0 para que funcionara para n=1 en Periodicity, es decir, para 2 compras

In [258]:
custom_features(df_retail)

,Customer ID,Lenght,Recency,Frequency,Monetary,Periodicity
0,12346.0,192.0,165,11,18.760000,34.778729
1,12347.0,37.0,3,2,81.475000,0.000000
2,12349.0,180.0,43,3,291.780000,72.000000
3,12352.0,16.0,11,2,27.300000,0.000000
4,12356.0,43.0,16,3,86.476667,9.500000
...,...,...,...,...,...,...
2892,18276.0,334.0,28,5,36.594000,32.345788
2893,18277.0,97.0,33,4,74.367500,33.209771
2894,18283.0,273.0,18,6,79.545000,55.304973
2895,18286.0,247.0,112,2,143.150000,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [259]:
class MinMax(BaseEstimator, TransformerMixin):
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        X_=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        return X_

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [260]:
function_custom_features=FunctionTransformer(custom_features)
scaler=ColumnTransformer(transformers=[('Escalamiento MinMax',MinMax(),['Lenght','Recency','Frequency','Monetary','Periodicity'])])
pipe_tsne=Pipeline([('Creación LRMFP',function_custom_features),('Escalamiento',scaler),('TSNE',TSNE(random_state=27))])

In [261]:
proy=pipe_tsne.fit_transform(df_retail)

In [262]:
proy

array([[-34.33893  , -20.741325 ],
       [-38.65664  ,  44.894928 ],
       [ 13.549181 ,  26.285341 ],
       ...,
       [ 28.641579 ,   7.6547885],
       [-21.534506 , -29.691824 ],
       [  8.476948 ,  22.002632 ]], dtype=float32)

In [263]:
fig_df = pd.concat([
    custom_features(df_retail).loc[:, ['Customer ID']],
    MinMax().transform(custom_features(df_retail).iloc[:, 1:]),
    pd.DataFrame(proy, columns=["x", "y"])
    
    ],axis=1)

fig_df

,Customer ID,Lenght,Recency,Frequency,Monetary,Periodicity,x,y
0,12346.0,0.516129,0.439678,0.044118,0.002650,0.192680,-34.338928,-20.741325
1,12347.0,0.099462,0.005362,0.000000,0.011664,0.000000,-38.656639,44.894928
2,12349.0,0.483871,0.112601,0.004902,0.041889,0.398892,13.549181,26.285341
3,12352.0,0.043011,0.026810,0.000000,0.003878,0.000000,-47.139236,47.386181
4,12356.0,0.115591,0.040214,0.004902,0.012383,0.052632,-30.690372,44.582878
...,...,...,...,...,...,...,...,...
2892,18276.0,0.897849,0.072386,0.014706,0.005213,0.179201,46.785728,-18.528139
2893,18277.0,0.260753,0.085791,0.009804,0.010642,0.183988,-13.169206,33.757038
2894,18283.0,0.733871,0.045576,0.019608,0.011386,0.306399,28.641579,7.654788
2895,18286.0,0.663978,0.297587,0.000000,0.020528,0.000000,-21.534506,-29.691824


In [264]:
def get_scatter(fig_df, color_col):
    fig = px.scatter(
        fig_df,
        x="x",
        y="y",
        color=color_col,
        range_x=(fig_df["x"].min() - 1, fig_df["x"].max() + 1),
        range_y=(fig_df["y"].min() - 1, fig_df["y"].max() + 1),
    )
    return fig


get_scatter(fig_df, "Lenght")

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [265]:
pipe_kmeans=Pipeline([('Creación LRMFP',function_custom_features),('Escalamiento',scaler),
                  ('Clustering',KMeans(random_state=27))])

inercias=np.zeros((20,2))
inercias[:,0]=np.arange(1,21)
for k in range(20):
    pipe_kmeans.set_params(Clustering__n_clusters=k+1)
    pipe_kmeans.fit(df_retail)
    inercias[k,1]=pipe_kmeans.named_steps['Clustering'].inertia_

inercias = pd.DataFrame(inercias, columns=["n° clusters", "inertia"])
inercias

,n° clusters,inertia
0,1.0,451.939052
1,2.0,212.992892
2,3.0,159.135472
3,4.0,121.328726
4,5.0,95.260582
5,6.0,84.198122
6,7.0,74.602091
7,8.0,66.426555
8,9.0,61.422987
9,10.0,56.497084


In [266]:
px.line(inercias, x="n° clusters", y="inertia", title="Método del Codo con K-Means",markers=True)

Se puede ver que el último cambio notable en la "inertia" se produce para $k=5$, pues después de este valor la inertia decrece casi a tasa constante, es decir, se parece mucho a una recta. Así, siguiendo la idea del método del codo, se elige $k=5$ como la cantidad óptima de clusters.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [267]:
n_clus=5
pipe_kmeans.set_params(Clustering__n_clusters=n_clus)
pipe_kmeans.fit(df_retail)
scaled_centers=pipe_kmeans.named_steps['Clustering'].cluster_centers_
data=custom_features(df_retail).iloc[:,1:]
clusters,size_clusters=np.unique(pipe_kmeans.fit_predict(df_retail),return_counts=True)

In [268]:
df_clusters=pd.DataFrame(scaled_centers,columns=['Lenght','Recency','Frequency','Monetary','Periodicity'])
df_clusters['Tamaño del cluster']=size_clusters
df_clusters

# tabla con valores normalizados

,Lenght,Recency,Frequency,Monetary,Periodicity,Tamaño del cluster
0,0.146499,0.106707,0.004581,0.011124,0.030529,595
1,0.522244,0.157868,0.013004,0.009458,0.130986,815
2,0.868594,0.054121,0.049943,0.009705,0.139682,811
3,0.207005,0.584396,0.004196,0.011929,0.049512,347
4,0.761244,0.095894,0.011224,0.010863,0.474115,329


In [269]:
centers=scaled_centers.copy()
for i in range(n_clus):
    centers[i,:]=centers[i,:]*(data.max(axis=0)-data.min(axis=0))+data.min(axis=0)
df_clusters2=pd.DataFrame(centers,columns=['Lenght','Recency','Frequency','Monetary','Periodicity'])
df_clusters2['Tamaño del cluster']=size_clusters
df_clusters2

# tabla con valores en escala real

,Lenght,Recency,Frequency,Monetary,Periodicity,Tamaño del cluster
0,54.497479,40.801681,2.934454,77.717386,5.510430,595
1,194.274847,59.884663,4.652761,66.129832,23.643047,815
2,323.116995,21.187192,12.188424,67.843368,25.212528,811
3,77.005764,218.979827,2.855908,83.318404,8.936950,347
4,283.182927,36.768293,4.289634,75.901749,85.577713,329


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

Se puede observar que los clientes del Cluster 2 podrían ser considerados como clientes "fieles" de la empresa, pues se puede ver que este cluster tiene un promedio de "Lenght" grande, es decir, entre la primera y última compra hay hartos días, su "Recency" es más pequeño, lo que quiere decir que la última compra no fue hace hartos días, y su "Frequency" también es mayor que para los otros clusters, lo que quiere decir que los clientes de este cluster son los que más han comprado en a tienda. Así, este cluster representa a los clientes que compran más en la tienda, y desde hace más tiempo.

Por otro lado, el cluster 4 tiene un "Lenght" promedio grande, pero una "Frequency" bastante menor que el cluster 2. Así, este tipo de personas se puede interpretar como clientes que conocen la tienda desde hace tiempo (y compran desde hace tiempo), pero sólo compran esporádicamente, lo que se puede confirmar aún más observando la gran "Periodicity" promedio de este cluster.

El cluster 1 tiene un "Lenght" relativament grande, pero mejor al de los 2 clusters anteriores, un "Recency" mayor (es decir, la última compra fue realizada hace más tiempo), una "Frequency" similar al cluster 4, pero una "Periodicity" bastante menor. Así, este tipo de clientes se puede interpretar como uno que compra poco en la empresa, pero suele hacerlo más periódicamente (cada cierto intervalo de tiempo más o menos regular, el cliente compra en la tienda, pero este intervalo es relativamente grande).

Las personas del cluster 0 se pueden interpretar como nuevos clientes de la empresa, que con el tiempo pueden ser considerados como fieles. Esto pues tanto el "Lenght", el "Recency", la "Frequency" y la "Periodicity" promedio son relativamente pequeños en comparación a los valores que se pueden encontrar en cada variable, por lo que se puede suponer que este tipo de clientes lleva poco tiempo comprando en la tienda, pero lo ha hecho relativamente reciente, y de manera regular. 

Por último, el cluster 3 tiene un "Lenght" promedio no muy grande, pero una gran "Recency". Así, se puede interpretar que el tipo de cliente de este cluster es un cliente que la empresa "ya perdió", pues compró durante un corto período de tiempo ("Lenght" relativamente pequeño) pero hace mucho tiempo ("Recency" grande), lo que implica una "Frequency" y una "Periodicity" pequeñas.

Acerca de la variable "Monetary" no se puede decir mucho, pues el promedio de cada cluster es similar. Esto se puede explicar en que las otras variables describen más el comportamiento de los clientes para cada cluster (como se explicó), pero dentro de cada tipo de cliente existen personas con "Monetary" de diferentes tipos, lo que genera un promedio similar.

Con todo esto, se concluye que las segmentaciones generadas hacen sentido y parecen coherentes.



#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [270]:
fig_df2 = pd.concat([
    custom_features(df_retail).loc[:, ['Customer ID']],
    MinMax().transform(custom_features(df_retail).iloc[:, 1:]),
    pd.DataFrame(proy, columns=["x", "y"]),
    pd.DataFrame(pipe_kmeans.fit_predict(df_retail),columns=['Cluster'])
    ],axis=1)


get_scatter(fig_df2, "Cluster")

Los clusters se ven separados en el sentido de que tienden a no solaparse, salvo por algunos datos en las fronteras entre dos clusters, pero no se ven tan separados en el sentido de "distancia entre clusters". Así, se puede concluir que los clusters se ven bien agrupados y se pueden identificar en el gráfico sin mezclarse entre ellos, pero sus fronteras no son lejanas.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>